# 模型训练流程
* 1.Data provider 
    * a.Image data
    * b.random vector  
* 2.Build Compute graph  
    * a.generator 
    * b.discriminator 
    * c.DCGAN class 
* 3.training process 

In [1]:
import os
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import sys
import pprint
import numpy as np
import pickle
import random
import math
from PIL import Image
from tensorflow.examples.tutorials.mnist import input_data

D:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('../MNIST_data/', one_hot=True)# 加载数据
output_dir = './local_run' # 输出文件夹路径
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir) # 创建输出文件夹

def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100, # 随机噪声数据维度
        init_conv_size = 4, # 将输入噪声变化成feature_map时,其初始size为4x4
        g_channels = [128, 64, 32, 1],# 生成器中各反卷积层的通道数目
        d_channels = [32, 64, 128, 256], # 判别器中各卷积层的通道数据,各个卷积层size减半,然后通道数加半,然后pooling
        batch_size = 128, # 每批数量
        learning_rate = 0.002, 
        beta1 = 0.5, # adam参数
        img_size = 32, # 生成图像为32x32
    )
hparams = get_default_params()


Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
# data provider
class MnistData(object):
    # z_dim随机向量的长度, img_size,将train中的img resize成img_size
    def __init__(self, mnist_train, z_dim, img_size):
        self._data = mnist_train
        self._example_num = len(self._data)
        # 随机向量使用正态分布生成
        self._z_data = np.random.standard_normal((self._example_num, z_dim)) 
        # next batch 指示器
        self._indicator = 0
        # resize 图像大小,参数为要resize到的图像大小
        self._resize_mnist_img(img_size)
        # shuffle函数
        self._random_shuffle()
        
    def _random_shuffle(self):
        x = np.random.permutation(self._example_num)
        # 随机化train和z
        self._data = self._data[x]
        self._z_data = self._z_data[x]  
    
    def _resize_mnist_img(self, img_size):
        """resize the img to target imgsize,first numpy to PIL img"""
        # 1.在mnist中图像都被归一化,所以先将图像灰度级恢复到0~255,像素值类型为整形
        data = np.asarray(self._data * 255, np.uint8)
        # 2. 像素数据 [example_num, 784] -> [example_num, 28, 28]
        data = data.reshape((self._example_num, 28, 28))
        # 3.将每幅图像插值成img_size * img_size
        new_data = []
        for i in range(self._example_num):
            img = data[i]
            # 将numpy数据变成PIL对象
            img = Image.fromarray(img)
            img = img.resize((img_size, img_size))
            # 再将PTL对象变成numpy对象
            img = np.asarray(img)
            # 判别器神经网络需要图像通道信息
            img = img.reshape((img_size,img_size, 1))
            new_data.append(img)
        # 将resize后的全部图片转成numpy类型,new_data这里还是一个字典
        new_data = np.asarray(new_data, dtype=np.float32)
        # 将new_data归一化,这里将像素数据归一化到-1~1直接,为了配合generator,G中使用的是tanH
        new_data = new_data / 127.5 - 1 
        # self._data [example_num, img_size, img_size ,1]
        self._data = new_data  
        
    def next_batch(self, batch_size):
        """下一batch"""
        end_indicator = self._indicator + batch_size  
        if end_indicator > self._example_num: 
            # 如果end超出数据范围,直接shuffle,重新划分
            self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self._example_num
        
        batch_data = self._data[self._indicator : end_indicator]
        batch_z = self._z_data[self._indicator : end_indicator]
        # 将当前indicator置为end_indicator,以便下一批次数据划分
        self._indicator = end_indicator
        return batch_data, batch_z 

mnist_data = MnistData(mnist.train.images, hparams.z_dim, hparams.img_size)
batch_data, batch_z = mnist_data.next_batch(5)

In [10]:
# generator 
def conv2d_transpose(inputs, out_channel, name, training, with_bn_relu=True):
    """
    Wrapper of conv2d transpose
    out_channel:输出图像通道
    name:该scope命名空间
    training:在做卷积或者反卷积时都需要加一个bn操作,这里作为一个training开关
    with_bn_relu=True:bn操作不作用于输出层和输入层,而且在生成器中其他层都使用relu激活,输出层使用tanH
    所以这里默认为True,当到输出层时将该开关关闭
    """
    conv2d_trans = tf.layers.conv2d_transpose(inputs,
                                             out_channel,
                                             [5,5],
                                             strides=(2,2),
                                             padding='SAME')
    if with_bn_relu:
        bn = tf.layers.batch_normalization(conv2d_trans, training=training)
        return tf.nn.relu(bn)
    else:
        return conv2d_trans
    
class Generator(object):
    def __init__(self, channels, init_conv_size):
        """
        channels:各个反卷积层中的通道数
        """
        self._channels = channels
        self._init_conv_size = init_conv_size
        # reuse 开关,在第一次构建计算图完成后,就将该重用标志打开
        self._reuse = False
        
    def __call__(self, inputs, training):
        # 首先将输入数据转成tensor类型
        inputs = tf.convert_to_tensor(inputs)
        with tf.variable_scope('generator', resue=self._reuse):
            """
            1.inputs -> fc 
            2.
            """
        # 在第一次计算图构建完成后,将resue打开
        self._reuse = True
        
    
        